In [145]:
from IPython.core.display import HTML
with open('../style.css', 'r') as file:
    css = file.read()
HTML(css)

# Die Onkologie-Station

Auf einer Onkologie-Station liegen fünf Patienten in nebeneinander liegenden Zimmern.
Bis auf einen  der Patienten hat jeder genau eine Zigaretten-Marke geraucht.
Der Patient, der nicht Zigarette geraucht hat, hat Pfeife geraucht.
Jeder Patient fährt genau ein Auto und ist
an genau einer Krebs-Art erkrankt.  Zusätzlich haben Sie die folgenden Informationen:
<ol>
<li> Im Zimmer neben Michael wird Camel geraucht. </li>
<li> Der Trabant-Fahrer raucht Ernte 23 und liegt im Zimmer neben dem 
      Zungen-Krebs Patienten. </li>
<li> Rolf liegt im letzten Zimmer und hat Kehlkopf-Krebs. </li>
<li> Der West-Raucher liegt im ersten Zimmer. </li>
<li> Der Mazda-Fahrer hat Zungen-Krebs und liegt neben dem Trabant-Fahrer. </li>
<li> Der Nissan-Fahrer liegt neben dem Zungen-Krebs Patient. </li>
<li> Rudolf wünscht sich Sterbe-Hilfe und liegt zwischen dem Camel-Raucher und dem Trabant-Fahrer. </li>
<li> Der Seat Fahrer hat morgen seinen letzten Geburtstag. </li>
<li> Der Luckies Raucher liegt neben dem Patienten mit Lungen-Krebs. </li>
<li> Der Camel Raucher liegt neben dem Patienten mit Darm-Krebs. </li>
<li> Der Nissan Fahrer liegt neben dem Mazda-Fahrer. </li>
<li> Der Mercedes-Fahrer raucht Pfeife und liegt neben dem Camel Raucher. </li>
<li> Jens liegt neben dem Luckies Raucher. </li>
<li> Der Hodenkrebs-Patient hat gestern seine Eier durchs Klo gespült. </li>
</ol>
Entwickeln Sie ein <em>Python</em>-Programm, das die folgenden Fragen beantwortet:
<ol> 
<li> Was raucht der Darmkrebs-Patient? </li>
<li> Was fährt Kurt für ein Auto? </li>
</ol>

## Importing the Necessary Modules

Our goal is to solve this puzzle by first coding it as a solvability problem of propositional logic and then to solve the resulting set of clauses using the algorithm of Davis and Putnam.

In [146]:
import davisPutnam as dp

In order to be able to transform formulas from propositional logic into sets of clauses we import the module <tt>cnf</tt> which implements the function <tt>normalize</tt> that takes a formula and transforms it into a set of clauses.

In [147]:
import cnf

In order to write formulas conveniently, we use the parser for propositional logic.

In [148]:
import propLogParser as plp

Using the parser and the module <tt>cnf</tt> we can impement a function $\texttt{parseCNF}(s)$ that takes a string $s$ representing a formula and transforms $s$ into an equivalent set of clauses.

In [149]:
def parseCNF(s):
    nestedTuple = plp.LogicParser(s).parse()
    Clauses     = cnf.normalize(nestedTuple)
    return Clauses

## Auxiliary Functions

In [150]:
def atMostOne(V):
    return { frozenset({ ('¬', p), ('¬', q) })
            for p in V
            for q in V
                if p != q }
atMostOne({ "a", "b", "c" })

{frozenset({('¬', 'b'), ('¬', 'c')}),
 frozenset({('¬', 'a'), ('¬', 'c')}),
 frozenset({('¬', 'a'), ('¬', 'b')})}

Given a name $f$ and an index $i \in\{1,2,3,4,5\}$, the function $\texttt{var}(i)$ creates the string 
$f\langle i \rangle$, e.g. the call <tt>var("Japanese", 2)</tt> returns the following string:

<tt>Japanese$\langle$2$\rangle$</tt>.

In [151]:
def var(f, i):
    return f + "<" + str(i) + ">" 

In [152]:
var("Japanese", 2)

'Japanese<2>'

The call $\texttt{flatten}(\texttt{LoS})$ takes list of sets $\texttt{LoS}$ and adds all the sets in this list into one big set. 

In [153]:
def flatten(ListOfSets):
    return {x for S in ListOfSets for x in S}

In [154]:
flatten([{1,2,3}, {3,4,5}])

{1, 2, 3, 4, 5}

A call of the form $\texttt{x}$ will return a clause that specifies that the person with property $x$ has to live in one of the houses from $1$ to $5$.

In [155]:
def somewhere(x):
    return frozenset({ var(x, i) for i in range(1, 5+1) })

In [156]:
somewhere("a")

frozenset({'a<1>', 'a<2>', 'a<3>', 'a<4>', 'a<5>'})

Given an exclusive set of properties $S$ and a house number $i$, the function $\texttt{atMostOne}(S, i)$ returns a set of clauses that specifies that the person living in house number $i$ has at most one of the properties from the set $S$.  For example, if 
$S = \{\texttt{"Japanese"}, \texttt{"Englishman"}, \texttt{"Spaniard"}, \texttt{"Norwegian"}, \texttt{"Ukranian"}\}$, 
then $\texttt{atMostOne}(S, 3)$ specifies that the inhabitant of house number 3 has at most one of the nationalities from the set $S$.

In [157]:
def atMostOneAt(S, i):
    return atMostOne({ var(s, i) for s in S })

In [158]:
atMostOneAt({"A", "B", "C"}, 1)

{frozenset({('¬', 'B<1>'), ('¬', 'C<1>')}),
 frozenset({('¬', 'A<1>'), ('¬', 'B<1>')}),
 frozenset({('¬', 'A<1>'), ('¬', 'C<1>')})}

Implement a function $\texttt{onePerHouse}(S)$ which could be called as follows:
$$\texttt{onePerHouse}(\{\texttt{"Japanese"},
       \texttt{"Englishman"}, 
       \texttt{"Spaniard"}, \texttt{"Norwegian"}, 
       \texttt{"Ukranian"}\})
$$
This function would create a set of clauses that expresses that there has to be a house where the Japanese lives, a house where the Englishman lives, a house where the Spaniard lives, a house where the Norwegian lives, and a house
where the Ukranian lives.  Furthermore, the set of clauses would create clauses that express that these five persons live in **different** houses.

When implementing this function, you should use the functions <tt>somewhere</tt> and <tt>atMostOne</tt>.

In [159]:
def onePerRoom(S):
    return { somewhere(s) for s in S } | { a for i in range(1, 5+1) for a in atMostOneAt(S, i) }

In [160]:
onePerRoom({"A", "B", "C", "D", "E"})

{frozenset({('¬', 'C<3>'), ('¬', 'E<3>')}),
 frozenset({('¬', 'B<4>'), ('¬', 'D<4>')}),
 frozenset({('¬', 'A<4>'), ('¬', 'D<4>')}),
 frozenset({('¬', 'B<5>'), ('¬', 'C<5>')}),
 frozenset({('¬', 'A<1>'), ('¬', 'D<1>')}),
 frozenset({('¬', 'A<3>'), ('¬', 'C<3>')}),
 frozenset({('¬', 'A<5>'), ('¬', 'D<5>')}),
 frozenset({('¬', 'D<5>'), ('¬', 'E<5>')}),
 frozenset({'B<1>', 'B<2>', 'B<3>', 'B<4>', 'B<5>'}),
 frozenset({('¬', 'A<1>'), ('¬', 'B<1>')}),
 frozenset({('¬', 'D<1>'), ('¬', 'E<1>')}),
 frozenset({('¬', 'D<4>'), ('¬', 'E<4>')}),
 frozenset({'C<1>', 'C<2>', 'C<3>', 'C<4>', 'C<5>'}),
 frozenset({'E<1>', 'E<2>', 'E<3>', 'E<4>', 'E<5>'}),
 frozenset({('¬', 'A<3>'), ('¬', 'B<3>')}),
 frozenset({('¬', 'B<5>'), ('¬', 'D<5>')}),
 frozenset({'A<1>', 'A<2>', 'A<3>', 'A<4>', 'A<5>'}),
 frozenset({('¬', 'B<1>'), ('¬', 'E<1>')}),
 frozenset({('¬', 'D<3>'), ('¬', 'E<3>')}),
 frozenset({('¬', 'C<4>'), ('¬', 'E<4>')}),
 frozenset({('¬', 'B<3>'), ('¬', 'E<3>')}),
 frozenset({('¬', 'A<2>'), ('¬', 'C<

Given to properties $a$ and $b$ the function $\texttt{sameHouse}(a, b)$ computes a set of clauses that specifies that if the inhabitant of house number $i$ has the property $a$, then he also has the property $b$ and vice versa.  For example, $\texttt{sameHouse}(\texttt{"Japanese"}, \texttt{"Dog"})$ specifies that the Japanese guy keeps a dog.

In [161]:
def sameRoom(a, b):
    return { p for i in range(1, 5+1) for p in parseCNF(f'{var(a, i)} ↔ {var(b, i)}') }

In [162]:
sameRoom("Luckies", "Camel")

{frozenset({('¬', 'Camel<3>'), 'Luckies<3>'}),
 frozenset({('¬', 'Luckies<3>'), 'Camel<3>'}),
 frozenset({('¬', 'Camel<2>'), 'Luckies<2>'}),
 frozenset({('¬', 'Luckies<1>'), 'Camel<1>'}),
 frozenset({('¬', 'Luckies<4>'), 'Camel<4>'}),
 frozenset({('¬', 'Camel<4>'), 'Luckies<4>'}),
 frozenset({('¬', 'Camel<1>'), 'Luckies<1>'}),
 frozenset({('¬', 'Luckies<2>'), 'Camel<2>'}),
 frozenset({('¬', 'Luckies<5>'), 'Camel<5>'}),
 frozenset({('¬', 'Camel<5>'), 'Luckies<5>'})}

Given to properties $a$ and $b$ the function $\texttt{nextTo}(a, b)$ computes a set of clauses that specifies that the inhabitants with properties $a$ and $b$ are direct neighbours.  For example, $\texttt{nextTo}(\texttt{'Japanese'}, \texttt{'Dog'})$ specifies that the Japanese guy lives next to the guy who keeps a dog.

In [163]:
def differentRoom(a, b):
    return { frozenset({ ('¬', var(a, i)), ('¬', var(b, i)) }) for i in range(1, 5+1) }

In [164]:
differentRoom('A', 'B')

{frozenset({('¬', 'A<3>'), ('¬', 'B<3>')}),
 frozenset({('¬', 'A<5>'), ('¬', 'B<5>')}),
 frozenset({('¬', 'A<4>'), ('¬', 'B<4>')}),
 frozenset({('¬', 'A<1>'), ('¬', 'B<1>')}),
 frozenset({('¬', 'A<2>'), ('¬', 'B<2>')})}

In [165]:
def nextTo(a, b):
    result = set()
    
    for i in range(1, 5+1):
        if i != 1 and i != 5:
            result |= parseCNF(f"{var(a, i)} → {var(b, i+1)} ∨ {var(b, i-1)}") 
        
        
        elif i == 1:
            result |= parseCNF(f"{var(a, i)} → {var(b, i+1)}")
            
        elif i == 5:
            result |= parseCNF(f"{var(a, i)} → {var(b, i-1)}")
    
    return result

In [166]:
nextTo('A', 'B')

{frozenset({('¬', 'A<2>'), 'B<1>', 'B<3>'}),
 frozenset({('¬', 'A<3>'), 'B<2>', 'B<4>'}),
 frozenset({('¬', 'A<4>'), 'B<3>', 'B<5>'}),
 frozenset({('¬', 'A<5>'), 'B<4>'}),
 frozenset({('¬', 'A<1>'), 'B<2>'})}

In [167]:
Brands   = { "Camel", "Ernte", "West", "Luckies", "Pfeife" }
Cars     = { "Trabant", "Mazda", "Nissan", "Seat", "Mercedes" }
Cancers  = { "Zunge", "Kehlkopf", "Lunge", "Darm", "Hoden" }
Names    = { "Michael", "Rolf", "Rudolf", "Jens", "Kurt" }

In [168]:
def allClauses():
    # In jedem Zimmer wird genau eine Zigaretten-Marke geraucht.
    Clauses  = onePerRoom(Brands) #{ a for i in range(1, 5+1) for a in atMostOneAt(Brands, i) }
    # Jeder Patient hat genau ein Auto.
    Clauses |= onePerRoom(Cars) #{ a for i in range(1, 5+1) for a in atMostOneAt(Cars, i) }
    # Jeder Patient ist an genau einer Krebsart erkrankt.
    Clauses |= onePerRoom(Cancers)
    # Jeder Patient hat genau einen Vornamen.
    Clauses |= onePerRoom(Names)
    # Im Zimmer neben Michael wird Camel geraucht. 
    Clauses |= nextTo("Michael", "Camel")
    # Der Trabant-Fahrer raucht Ernte 23 und liegt im Zimmer neben dem 
    # Zungen-Krebs Patienten. 
    Clauses |= sameRoom("Trabant", "Ernte")
    Clauses |= nextTo("Trabant", "Zunge")
    # Rolf liegt im letzten Zimmer und hat Kehlkopf-Krebs. 
    Clauses |= parseCNF(f'{var("Rolf", 5)}')
    Clauses |= parseCNF(f'{var("Kehlkopf", 5)}')
    # Der West-Raucher liegt im ersten Zimmer. 
    Clauses |= parseCNF(f'{var("West", 1)}')
    # Der Mazda-Fahrer hat Zungen-Krebs und liegt neben dem Trabant-Fahrer. 
    Clauses |= sameRoom("Mazda", "Zunge")
    Clauses |= nextTo("Mazda", "Trabant")
    # Der Nissan-Fahrer liegt neben dem Zungen-Krebs Patient. 
    Clauses |= nextTo("Nissan", "Zunge")
    # Rudolf wünscht sich Sterbe-Hilfe und liegt zwischen dem Camel-Raucher und dem Trabant-Fahrer. 
    #Clauses |= sameRoom("Rudolf", "SterbeHilfe")
    Clauses |= nextTo("Rudolf", "Camel")
    Clauses |= nextTo("Rudolf", "Trabant")
    # Der Luckies Raucher liegt neben dem Patienten mit Lungen-Krebs. 
    Clauses |= nextTo("Luckies", "Lunge")
    # Der Camel Raucher liegt neben dem Patienten mit Darm-Krebs. 
    Clauses |= nextTo("Camel", "Darm")
    # Der Nissan Fahrer liegt neben dem Mazda-Fahrer. 
    Clauses |= nextTo("Nissan", "Mazda")
    # Der Mercedes-Fahrer raucht Pfeife und liegt neben dem Camel Raucher. 
    Clauses |= sameRoom("Mercedes", "Pfeife")
    Clauses |= nextTo("Mercedes", "Camel")
    # Jens liegt neben dem Luckies Raucher. 
    Clauses |= nextTo("Jens", "Luckies")
    return Clauses

In [169]:
Clauses = allClauses()
Clauses

{frozenset({('¬', 'Mazda<3>'), ('¬', 'Trabant<3>')}),
 frozenset({('¬', 'Darm<1>'), ('¬', 'Kehlkopf<1>')}),
 frozenset({('¬', 'Mercedes<3>'), ('¬', 'Trabant<3>')}),
 frozenset({'Nissan<1>', 'Nissan<2>', 'Nissan<3>', 'Nissan<4>', 'Nissan<5>'}),
 frozenset({('¬', 'Mazda<2>'), ('¬', 'Seat<2>')}),
 frozenset({('¬', 'Camel<1>'), 'Darm<2>'}),
 frozenset({('¬', 'Darm<3>'), ('¬', 'Kehlkopf<3>')}),
 frozenset({('¬', 'Mercedes<5>'), ('¬', 'Nissan<5>')}),
 frozenset({('¬', 'Kurt<2>'), ('¬', 'Michael<2>')}),
 frozenset({('¬', 'Mazda<3>'), ('¬', 'Nissan<3>')}),
 frozenset({('¬', 'Mazda<4>'), ('¬', 'Trabant<4>')}),
 frozenset({('¬', 'Nissan<1>'), ('¬', 'Seat<1>')}),
 frozenset({('¬', 'Mazda<3>'), 'Zunge<3>'}),
 frozenset({('¬', 'Mazda<1>'), ('¬', 'Mercedes<1>')}),
 frozenset({('¬', 'SterbeHilfe<4>'), 'Rudolf<4>'}),
 frozenset({('¬', 'Luckies<5>'), 'Lunge<4>'}),
 frozenset({('¬', 'Hoden<2>'), ('¬', 'Kehlkopf<2>')}),
 frozenset({'Darm<1>', 'Darm<2>', 'Darm<3>', 'Darm<4>', 'Darm<5>'}),
 frozenset({('¬'

There should be 322 different clauses.

In [170]:
len(Clauses)

318

In [171]:
def solve():
    Clauses = allClauses()
    return dp.solve(Clauses, set())

Solving the problem takes about 3 seconds on my computer.

In [172]:
%%time
Solution = solve()

CPU times: user 179 ms, sys: 3 ms, total: 182 ms
Wall time: 180 ms


In [173]:
Solution

{frozenset({('¬', 'Pfeife<5>')}),
 frozenset({('¬', 'Lunge<1>')}),
 frozenset({('¬', 'Kehlkopf<2>')}),
 frozenset({'Michael<2>'}),
 frozenset({('¬', 'Michael<4>')}),
 frozenset({('¬', 'Kurt<5>')}),
 frozenset({'Kehlkopf<5>'}),
 frozenset({'Nissan<3>'}),
 frozenset({('¬', 'Darm<4>')}),
 frozenset({'Trabant<5>'}),
 frozenset({('¬', 'Jens<4>')}),
 frozenset({('¬', 'Michael<3>')}),
 frozenset({'Pfeife<2>'}),
 frozenset({('¬', 'Mazda<5>')}),
 frozenset({('¬', 'Camel<2>')}),
 frozenset({('¬', 'Trabant<4>')}),
 frozenset({('¬', 'SterbeHilfe<1>')}),
 frozenset({('¬', 'Nissan<1>')}),
 frozenset({('¬', 'Luckies<2>')}),
 frozenset({('¬', 'Ernte<4>')}),
 frozenset({('¬', 'Trabant<2>')}),
 frozenset({'Jens<3>'}),
 frozenset({('¬', 'Seat<2>')}),
 frozenset({('¬', 'Seat<5>')}),
 frozenset({('¬', 'West<2>')}),
 frozenset({('¬', 'Rolf<1>')}),
 frozenset({('¬', 'Zunge<3>')}),
 frozenset({'Rudolf<4>'}),
 frozenset({('¬', 'Luckies<1>')}),
 frozenset({('¬', 'Mercedes<4>')}),
 frozenset({('¬', 'Darm<1>')}),

## Pretty Printing the Solution

In [174]:
from IPython.display import HTML

In [175]:
def arb(S):
    for x in S:
        return x

In [176]:
def extractAssignment(Solution):
    Assignment = {}
    for Unit in Solution:
        Literal = arb(Unit)
        if isinstance(Literal, str):
            number = int(Literal[-2])
            name   = Literal[:-3]
            Assignment[name] = number
    return Assignment

In [177]:
def showHTML(Solution):
    result  = '<table style="border:2px solid blue">\n'
    result += '<tr>'
    for name in ['Room', 'Brands', 'Cars', 'Cancers', 'Names']:
        result += '<th style="color:gold; background-color:blue">' + name + '</th>'
    result += '</tr>\n'
    for chair in range(1, 5+1):
        result += '<tr><td style="border:1px solid green">' + str(chair) + '</td>'
        for Class in [Brands, Cars, Cancers, Names]:
            for x in Class:
                if Solution[x] == chair:
                    result += '<td  style="border:1px solid green">' + x + '</td>'
        result += '</tr>\n'
    result += '</table>'
    display(HTML(result))

In [178]:
showHTML(extractAssignment(Solution))

Room,Brands,Cars,Cancers,Names
1,West,Seat,Hoden,Kurt
2,Pfeife,Mercedes,Darm,Michael
3,Camel,Nissan,Lunge,Jens
4,Luckies,Mazda,Zunge,Rudolf
5,Ernte,Trabant,Kehlkopf,Rolf


## Checking the Uniqueness of the Solution

Given a set of unit clauses $U$, the function $\texttt{checkUniqueness}(U)$ returns a clause that is the negation of the set $U$.

In [179]:
def negateSolution(UnitClauses):
    return { dp.complement(arb(unit)) for unit in UnitClauses }

In [180]:
negateSolution({ frozenset({'a'}), frozenset({('¬', 'b')}) }) 

{('¬', 'a'), 'b'}

The function $\texttt{checkUniqueness}(\texttt{Solution}, \texttt{Clauses})$  takes a set of $\texttt{Clauses}$ and a $\texttt{Solution}$ for these clauses and checks, whether this is the only solution.

In [181]:
def checkUniqueness(Solution, Clauses):
    negation = negateSolution(Solution)
    Clauses.add(frozenset(negation))
    alternative = dp.solve(Clauses, set())
    if alternative == { frozenset() }:
        print("Well done: The solution is unique!")
    else:
        print("ERROR: The solution is not unique!")

In [182]:
checkUniqueness(Solution, Clauses)

Well done: The solution is unique!
